# Remote Execute EHT job submission
1. login in osg node through ssh jump
2. execute the job submission script
3. monitoring and get the result back

## 1. ssh jump to osg node
osg node can be only accessed through scigap03

In [1]:
from fabric import Connection

In [4]:
user = "airavata"
sshA = "scigap03.sciencegateways.iu.edu"
sshB = "ospool-eht2000.chtc.wisc.edu"

In [2]:
# password can be securely stored somewhere 
import getpass
pw = getpass.getpass(prompt='password for the ssh key: ')

password for the ssh key:  ········


In [5]:
# connect to the first VM
c1 = Connection(sshA, user='airavata',connect_kwargs={'password': pw})
c1.run("hostname")

scigap03.sciencegateways.iu.edu


<Result cmd='hostname' exited=0>

In [6]:
# connect to the osg node
c2 = Connection(sshB, user='ehtbot', gateway=c1)
c2.run('hostname')

ospool-eht2000


<Result cmd='hostname' exited=0>

In [7]:
# check if any job running
c2.run("condor_q")



-- Schedd: ospool-eht.chtc.wisc.edu : <128.105.68.10:9618?... @ 02/15/24 09:57:50
OWNER BATCH_NAME      SUBMITTED   DONE   RUN    IDLE   HOLD  TOTAL JOB_IDS

Total for query: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended 
Total for ehtbot: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended 
Total for all users: 0 jobs; 0 completed, 0 removed, 0 idle, 0 running, 0 held, 0 suspended



<Result cmd='condor_q' exited=0>

## 2. Submit a simple job
ospool_test/eht_submit.py  
positional arguments:  
* ehtdata     ehtdata input files  
* rr          Rhigh-Ratio  
* tva         Theta-Viewing-Angle  
* rdm         rho0-density-normalization  

In [39]:
script = "ospool_test/eht_submit.py"
ehtdata = "torus.out0.05992.h5,torus.out0.05993.h5"
rr = "1"
tva = "30,50"
rdm = "4.266338570441294e+17"
submitcmd = f"{script} {ehtdata} {rr} {tva} {rdm}"
c2.run(submitcmd)

Split parameter file
Submit jobs
par/batch.p00 -> par/BATCH.D00
Submitting job(s)....
4 job(s) submitted to cluster 68.
DONE
[('torus.out0.05992.h5', '1', '30', '4.266338570441294e+17'), ('torus.out0.05992.h5', '1', '50', '4.266338570441294e+17'), ('torus.out0.05993.h5', '1', '30', '4.266338570441294e+17'), ('torus.out0.05993.h5', '1', '50', '4.266338570441294e+17')]
total:  4


<Result cmd='ospool_test/eht_submit.py torus.out0.05992.h5,torus.out0.05993.h5 1 30,50 4.266338570441294e+17' exited=0>

## 3. Monitor Job

In [52]:
c2.run("condor_q")



-- Schedd: ospool-eht.chtc.wisc.edu : <128.105.68.10:9618?... @ 02/15/24 10:36:33
OWNER  BATCH_NAME    SUBMITTED   DONE   RUN    IDLE  TOTAL JOB_IDS
ehtbot ID: 68       2/15 10:34      _      4      _      4 68.0-3

Total for query: 4 jobs; 0 completed, 0 removed, 0 idle, 4 running, 0 held, 0 suspended 
Total for ehtbot: 4 jobs; 0 completed, 0 removed, 0 idle, 4 running, 0 held, 0 suspended 
Total for all users: 4 jobs; 0 completed, 0 removed, 0 idle, 4 running, 0 held, 0 suspended



<Result cmd='condor_q' exited=0>

In [66]:
# check the errors .logs
c2.run("ls ospool_test/log")

img_s05992_Rh1_i30.log
img_s05992_Rh1_i50.log
img_s05993_Rh1_i30.err
img_s05993_Rh1_i30.log
img_s05993_Rh1_i30.out
img_s05993_Rh1_i50.log


<Result cmd='ls ospool_test/log' exited=0>

In [67]:
c2.run("more ospool_test/log/img_s05992_Rh1_i30.log")

::::::::::::::
ospool_test/log/img_s05992_Rh1_i30.log
::::::::::::::
000 (068.000.000) 2024-02-15 10:34:27 Job submitted from host: <128.105.68.10:9618?addrs=128.105.68.10-9618+[2607-f388-2200-100-216-3eff-fe11-1111]-9618&alias=ospool-eht.chtc.wisc.edu&noUDP&sock=schedd_2849159_ab6a>
...
040 (068.000.000) 2024-02-15 10:36:00 Started transferring input files
	Transferring to host: <131.225.208.52:44159?CCBID=128.104.103.162:9618%3faddrs%3d128.104.103.162-9618%26alias%3dospool-ccb.osg.chtc.io%26noUDP%26sock%3dcollector2#33971718%20192.170.231.9:9618%3faddrs%3d192.170.231.9-9618+[fd85-ee78-d8a6-8607--1-bcc0]-9618%26alias%3dospool-ccb.osgprod.tempest.chtc.io%26noUDP%26sock%3dcollector10#1898130&PrivNet=osg-1348949-0-fnpc22023.fnal.gov&addrs=131.225.208.52-44159+[2620-6a-0-8862-f0-0-208-52]-44159&alias=osg-1348949-0-fnpc22023.fnal.gov&noUDP>
...
040 (068.000.000) 2024-02-15 10:36:03 Finished transferring input files
...
021 (068.000.000) 2024-02-15 10:36:04 Message from starter on slot1_3@g

<Result cmd='more ospool_test/log/img_s05992_Rh1_i30.log' exited=0>

In [79]:
c2.run("condor_q")



-- Schedd: ospool-eht.chtc.wisc.edu : <128.105.68.10:9618?... @ 02/15/24 11:06:34
OWNER  BATCH_NAME    SUBMITTED   DONE   RUN    IDLE  TOTAL JOB_IDS
ehtbot ID: 68       2/15 10:34      3      1      _      4 68.1

Total for query: 1 jobs; 0 completed, 0 removed, 0 idle, 1 running, 0 held, 0 suspended 
Total for ehtbot: 1 jobs; 0 completed, 0 removed, 0 idle, 1 running, 0 held, 0 suspended 
Total for all users: 1 jobs; 0 completed, 0 removed, 0 idle, 1 running, 0 held, 0 suspended



<Result cmd='condor_q' exited=0>

In [80]:
# check the output
c2.run("ls ospool_test/out")

img_s05992_Rh1_i30.h5
img_s05993_Rh1_i30.h5
img_s05993_Rh1_i50.h5


<Result cmd='ls ospool_test/out' exited=0>

In [85]:
# close the connection
c2.close()
c1.close()